---
---
Problem Set 6: Web Scraping

Applied Data Science using Python

New York University, Abu Dhabi

Out: 7th Dec 2023 || **Due: 14th Dec 2023 at 23:59**

---
---
#Start Here
## Learning Goals
### General Goals
- Learn the fundamental concepts of Web Scraping

### Specific Goals
- Learn how to identify websites that allow scraping
- Learn the basics of BeautifulSoup
- Learn to adjust the scraping code to more than one query

## Collaboration Policy
- You are expected to comply with the [University Policy on Academic Integrity and Plagiarism](https://www.nyu.edu/about/policies-guidelines-compliance/policies-and-guidelines/academic-integrity-for-students-at-nyu.html).
- You are allowed to talk with / work with other students on homework assignments.
- You can share ideas but not code, analyses or results; you must submit your own code and results. All submitted code will be compared against all code submitted this semester and online using MOSS. We will also critically analyze the similarities in the submitted reports, methodologies, and results, **but we will not police you**. We expect you all to be mature and responsible enough to finish your work with full integrity.

## Late Submission Policy
You can submit the homework for upto 3 late days. However, we will deduct **20 points** from your homework grade **for each late day you take**. We will not accept the homework after 3 late days.

## Distribution of Class Materials
These problem sets and recitations are intellectual property of NYUAD, and we request the students to **not** distribute them or their solutions to other students who have not signed up for this class, and/or intend to sign up in the future. We also request you don't post these problem sets, and recitations online or on any public platforms.

## Disclaimer
The number of points do not necessarily signify/correlate to the difficulty level of the tasks.

## Submission
You will submit all your code as a Python Notebook through [Brightspace](https://brightspace.nyu.edu/) as **P6_YOUR NETID.ipynb**.

---

# General Instructions


This homework is worth 100 points. It has 4 parts. Below each part, we provide a set of concepts required to complete that part. All the parts need to be completed in a Jupyter (Colab) Notebook attached with this handout.

<font color="red">**Important Note:** Please scrape the websites ethically and make sure you utilize the `sleep` function between requests. </font>


# Part I: To Scrape or Not to Scrape (5 points)

A friend wants us to write a function that will scrape product data from 3 different websites, and will recommend which one to buy based on price and reviews. However, we need to figure out which of the below websites actually allow scraping:

- [Amazon.ae](https://www.amazon.ae/)
- [Noon.com](https://www.noon.com/uae-en/)
- [Instock.ae](https://www.instock.ae/)

Provide your answers, and, in a couple of sentences, describe how you have come to your answer.

*Hint: Two websites allow us to scrape their search results.*

Based on the analysis of the `robots.txt` files from the three websites, here are the conclusions regarding their scraping policies:

1. **[Amazon.ae](https://www.amazon.ae/)**: Amazon.ae's `robots.txt` file is quite restrictive. It disallows scraping in many parts of the website, including product, account, cart, and user interaction pages. Specific bots are entirely blocked. Thus, Amazon.ae does not generally allow scraping for product data.

2. **[Noon.com](https://www.noon.com/uae-en/)**: Noon.com's `robots.txt` file is more permissive. It only disallows access to paths under `/_svc/` while allowing all other paths. This indicates that Noon.com generally allows scraping, including for product data.

3. **[Instock.ae](https://www.instock.ae/)**: Instock.ae's `robots.txt` file restricts access to various parts of the site, including specific directories and functionalities related to products and customer interactions. However, it does not explicitly disallow scraping of product data from search results or general product pages.

Therefore, based on the `robots.txt` analysis, Noon.com and Instock.ae allow scraping of their search results to some extent, while Amazon.ae does not appear to permit scraping for product data.

### *Concepts required to complete this task:*

- Concepts of Ethical Scraping

## Rubric

- +5 points for correct answer and pointing the place you found the information

# Part II: Hindi Geet Mala (40 points)

[Hindi Geet Mala](https://www.hindigeetmala.net/) is a website containing information about Indian movies, songs, singers, etc. For this part, you will scrape information about all the movies in alphabetical order. Additionally, you will scrape information about songs.

More precisely, you will submit 2 CSV files:

* movies.csv: Title, Year, Number of Songs, Film Director, Film Producer, Film cast, Lyricist, Music Director, Singer, External links, Watch Full Movie

* songs.csv: Artists, Title, Rating, Number of Votes, Movie Title

**Notes:**

- you are NOT allowed to hardcode the list of letters.
- you are NOT allowed to use Pandas function `read_html()`

In [65]:
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
}

In [127]:
# Write you solution here
############# SOLUTION ###############
# Datframes to store the data (will be converted to CSV for the submission)
movies_df = pd.DataFrame(columns=['Title', 'Year', 'Number of Songs', 'Film Director', 'Film Producer', 'Film cast', 'Lyricist', 'Music Director', 'Singer', 'External links', 'Watch Full Movie'])
songs_df = pd.DataFrame(columns=['Artists', 'Title', 'Rating', 'Number of Votes', 'Movie Title'])

# Function to get movie URLs (relative) for each alphabet
def get_movie_urls(alphabet):
    url = f'https://www.hindilyrics4u.com/movie/{alphabet}.php'
    
    # Get the HTML content
    response = requests.get(url, headers=header)
    html = response.content
    
    # Parse the HTML
    soup = bs(html, "html.parser")
    
    # Get the movie table (class = bi and alcen)
    table = soup.find("table", attrs={"class": "b1", "class": "w760", "class": "alcen"})
    
    # Get all tds in the table that have a w25p class
    tds = table.find_all("td", attrs={"class": "w25p"})
    # print(len(tds))
    
    # Get all the links in the tds
    links = []
    for td in tds:
        links.append(td.find("a").get("href"))
    
    # Return the links
    return links

# Helper function to get title and year of movie from parsed HTML
def get_title_year(soup: bs):
    # Movie name and year can be found in a span with itemprop = inAlbum
    movie_name_year_span = soup.find("span", attrs={"itemprop": "inAlbum"})
    
    if not movie_name_year_span:
        return None, None
    
    # Movie name and year stored in a child a tag with itemprop = url
    movie_name_year_a = movie_name_year_span.find("a", attrs={"itemprop": "url"})
    
    # Movie name stored in a span with itemprop = name
    movie_name_span = movie_name_year_a.find("span", attrs={"itemprop": "name"})
    movie_name = movie_name_span.text
    
    # Movie year stored in the remaining text in the a tag
    movie_year = movie_name_year_a.text.replace(movie_name, '').strip()
    
    return movie_name, movie_year

def get_song_details(soup: bs):
    # Get the trs with itemprop = "track"
    trs = soup.find_all("tr", attrs={"itemprop": "track"})
    
    # Get the song details from each tr
    for tr in trs:
        # Get the song name
        song_name = tr.find("span", attrs={"itemprop": "name"}).text.strip()
        
        # Get the song rating
        song_rating = tr.find("span", attrs={"itemprop": "ratingValue"}).text.strip()
        
        # Get the song number of votes
        song_number_of_votes = tr.find("span", attrs={"itemprop": "ratingCount"}).text.strip()
        
        # Get the song artists
        song_artists = tr.find_all("span", attrs={"itemprop": "byArtist"})
        song_artists = [artist.text.strip() for artist in song_artists]
        
        # Get the movie title
        movie_title = soup.find("span", attrs={"itemprop": "inAlbum"}).find("a", attrs={"itemprop": "url"}).find("span", attrs={"itemprop": "name"}).text.strip()
        
        # Get the song details
        song_details = {
            'Artists': ', '.join(song_artists), # Convert the list to a comma separated string using the join function
            'Title': song_name,
            'Rating': song_rating,
            'Number of Votes': song_number_of_votes,
            'Movie Title': movie_title
        }
        
        # Append the song details to the songs dataframe global variable
        global songs_df
        songs_df = pd.concat([songs_df, pd.DataFrame(song_details, index=[0])])
    
    # Return the number of songs
    return len(trs)

def get_other_details(soup: bs):
    film_directors = []
    film_producers = []
    film_cast = []
    lyricists = []
    music_directors = []
    singers = []
    external_links = []
    watch_full_movie = []
    
    # Film directors stored in a td with itemprop = director
    film_directors_td = soup.find("td", attrs={"itemprop": "director"})
    if film_directors_td:
        film_director_spans = film_directors_td.find_all("span", attrs={"itemprop": "name"})
        for span in film_director_spans:
            film_directors.append(span.text)
        
    # Film producers stored in a td with itemprop = producer
    film_producers_td = soup.find("td", attrs={"itemprop": "producer"})
    if film_producers_td:
        film_producer_spans = film_producers_td.find_all("span", attrs={"itemprop": "name"})
        for span in film_producer_spans:
            film_producers.append(span.text)
    
    # Film cast stored in a td with itemprop = actor
    film_cast_td = soup.find("td", attrs={"itemprop": "actor"})
    if film_cast_td:
        film_cast_spans = film_cast_td.find_all("span", attrs={"itemprop": "name"})
        for span in film_cast_spans:
            film_cast.append(span.text)
    
    # Lyricists stored along with td with text = "Lyriscist: "
    lyricists_td = soup.find("td", text=re.compile("Lyricist: "))
    if lyricists_td:
        # Get the lyricists
        lyricists = lyricists_td.next_sibling.text.split(', ')

    # Do the same for singers
    singers_td = soup.find("td", text=re.compile("Singer: "))
    if singers_td:
        # Get the singers
        singers = singers_td.next_sibling.text.split(', ')
    
    # For music directors, we need to get the next td with itemprop = musicBy
    music_directors_td = soup.find("td", attrs={"itemprop": "musicBy"})
    if music_directors_td:
        music_director_spans = music_directors_td.find_all("span", attrs={"itemprop": "name"})
        for span in music_director_spans:
            music_directors.append(span.text)
    
    # External links stored in sibling of td with text = "External Links: "
    external_links_td = soup.find("td", text=re.compile("External Links: "))
    if external_links_td:
        external_links_td = external_links_td.next_sibling
        # Go through a tags in the td and get the href
        external_links_a = external_links_td.find_all("a")
        for a in external_links_a:
            external_links.append(a.get("href"))
    
    # Do the same for watch full movie
    watch_full_movie_td = soup.find("td", text=re.compile("Watch Full Movie: "))
    if watch_full_movie_td:
        watch_full_movie_td = watch_full_movie_td.next_sibling
        # Go through a tags in the td and get the href
        watch_full_movie_a = watch_full_movie_td.find_all("a")
        for a in watch_full_movie_a:
            watch_full_movie.append(a.get("href"))
            
    
    return {
        'Film Director': ', '.join(film_directors),
        'Film Producer': ', '.join(film_producers),
        'Film cast':  ', '.join(film_cast),
        'Lyricist':  ', '.join(lyricists),
        'Music Director':  ', '.join(music_directors),
        'Singer':  ', '.join(singers),
        'External links': ', '.join(external_links),
        'Watch Full Movie':  ', '.join(watch_full_movie)
    }
    

# Function to get movie details given the URL
def get_movie_details(url):
    # Movie name and year stored in an a tag with itemprop = url and href == url
    url = f'https://www.hindilyrics4u.com{url}'
    
    # Get the HTML content
    response = requests.get(url, headers=header)
    
    # Parse the HTML
    soup = bs(response.content, "html.parser")
    
    # Get the movie name and year
    movie_name, movie_year = get_title_year(soup)
    
    # Get the number of songs while appending the song details to the songs dataframe
    # This is a side effect of the get_song_details helper function
    number_of_songs = get_song_details(soup)
    
    # Get the other details
    other_details = get_other_details(soup)
    
    # Append the movie details to the movies dataframe global variable
    global movies_df
    movies_df = pd.concat([movies_df, pd.DataFrame({
        'Title': movie_name,
        'Year': movie_year,
        'Number of Songs': number_of_songs,
        **other_details
    }, index=[0])])
    

# Get the movie URLs for each alphabet
for alphabet in range(ord('a'), ord('z')+1):
    # Get the movie URLs for the alphabet
    movie_urls = get_movie_urls(chr(alphabet))
    
    # Get the movie details for each movie URL
    for movie_url in movie_urls:
        get_movie_details(movie_url)
        # Sleep for 1 second after scraping each movie
        time.sleep(1)
############# SOLUTION END ###############

In [122]:
# Save the dataframes to CSV
# Ensure that anything with commas is enclosed in quotes
movies_df.to_csv('movies.csv', quotechar='"', index=False)
songs_df.to_csv('songs.csv', quotechar='"', index=False)

### *Concepts required to complete this task:*

- Navigating through HTML code using functions
- DataFrame Creation and Writing to a file


## Rubric

- +30 points for correct output (15 points for each dataframe)
- +5 points for concise, logical code
- +3 points for ethical and mindful scraping
- +2 points for comments and variable names

# Part III: Yahoo Finance (30 points)

A friend who studies Economics, recommended us to buy stocks if the weekly average of closing price has been steadily increasing by at least 1% for the last 3 weeks.

Since we are interested in more than one company, let's write a function called `stock_decision()` that will scrape stock market close data for the given companies from [Yahoo Finance](https://finance.yahoo.com/), compute the weekly average for the past 3 weeks and check whether or not it has been increasing. If the weekly average has been steadily increasing by at least 1%, recommend to buy, otherwise recommend to sell.

The output should be two lines:

```
"Sell: "

"Buy: "
```

Implement the function for these companies.
- Apple
- Microsoft
- Amazon
- Tesla
- Facebook

For example, the url for Facebook would be https://finance.yahoo.com/quote/FB/history/.



**Hint:** For grouping the data per week, look up `resample()` function from Pandas. Another function that will come in handy is `pct_change()` from Pandas.

**Note:**

- you are NOT allowed to use Pandas function `read_html()`

In [30]:
# Write you solution here
############# SOLUTION ###############
def fetch_stock_data(symbol):
    # Create a stock prices dataframe with columns: Date, Open, High, Low, Close, Adj Close, Volume
    stock_data = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    
    # Construct URL for historical data of the stock symbol
    url = f"https://finance.yahoo.com/quote/{symbol}/history/"
    response = requests.get(url, headers=header)
    soup = bs(response.text, 'html.parser')    
    
    # Historical data is in a table with data-test attribute set to 'historical-prices'
    table = soup.find('table', attrs={'data-test': 'historical-prices'})
    tbody = table.find('tbody')
    rows = tbody.find_all('tr')
    
    # Iterate through each row and append the data to the stock_data dataframe
    for row in rows:
        # Each row has 7 columns
        cols = row.find_all('td')
        if len(cols) == 7:
            # Extract the text from each column and append to the stock_data dataframe
            stock_data = pd.concat([stock_data, pd.DataFrame([[cols[0].text, cols[1].text, cols[2].text, cols[3].text, cols[4].text, cols[5].text, cols[6].text]], columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])], ignore_index=True)

    return stock_data

def process_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Convert 'Close' to numeric, coerce errors
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Drop rows with NaN values (if any)
    df = df.dropna()

    # Resample to get weekly averages and calculate percentage change
    weekly_avg = df['Close'].resample('W').mean()
    pct_change = weekly_avg.pct_change()

    return pct_change

def stock_decision(companies):
    decisions = {'Buy': [], 'Sell': []}

    for company in companies:
        stock_data = fetch_stock_data(company)
        pct_change = process_data(stock_data)

        # Check if the weekly average has been steadily increasing by at least 1% for the last 3 weeks
        if all(pct_change[-3:] >= 0.01):
            decisions['Buy'].append(company)
        else:
            decisions['Sell'].append(company)

    return decisions

# Test the function with the specified companies
companies = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'FB']
# companies = ['SONO'] # Uncomment this line to test with a single company that is doing well enough to buy
decisions = stock_decision(companies)
print("Sell:", ", ".join(decisions['Sell']))
print("Buy:", ", ".join(decisions['Buy']))
############# SOLUTION END ###############

Sell: AAPL, MSFT, AMZN, TSLA, FB
Buy: 


### *Concepts required to complete this task:*

- Navigating through HTML code using functions
- DataFrame Creation and Manipulations
- Applying functions to a DataFrame


## Rubric

- +20 points for correct output
- +5 points for concise and logical code
- +3 points for ethical and mindful scraping
- +2 points for comments and logical variable names

# Part IV: WikiCFP (25 points)

[WikiCFP](https://wikicfp.com/) is a semantic wiki for Call for Papers in science and technology fields.

For this part, you will be writing a program that will do the following:

* scrape the first 10 pages of WikiCFP for the keyword `computer science`
* create a Pandas DataFrame with the columns `abbreviation, name, dates, place, deadline`
* create a new column called `country`, which will take the value `online` if the place contains `Online` or `Virtual`.
* remove missing values, and calculate the distribution of places and print the top and bottom countries.

**Note:**

- you are NOT allowed to use Pandas function `read_html()`

In [63]:
# Write you solution here
############# SOLUTION ###############
def scrape_wikicfp(keyword, pages=1):
    base_url = 'http://www.wikicfp.com/cfp/call?conference='
    df = pd.DataFrame(columns=['abbreviation', 'name', 'dates', 'place', 'deadline'])

    for page in range(1, pages + 1):
        url = f"{base_url}{keyword}&page={page}"
        time.sleep(1) # Some etiquette to not overload the server
        response = requests.get(url, headers=header)
        soup = bs(response.text, 'html.parser')
        
        events_table = soup.find('table', attrs={'cellpadding': '3', 'cellspacing': '1', 'width': '100%'})
        
        # Now, find TRs with bgcolor attribute set to '#f6f6f6' and '#e6e6e6'
        event_details_trs = events_table.find_all('tr', attrs={'bgcolor': ['#f6f6f6', '#e6e6e6']})
        
        # Now, split the event_details_trs into chunks of 2
        event_details = [event_details_trs[i:i+2] for i in range(0, len(event_details_trs), 2)]
        
        # Now, inside each event_details, find the tds with ONLY align attribute set to 'left'
        for event in event_details:
            first_row = event[0].find_all('td', attrs={'align': 'left'})
            second_row = event[1].find_all('td', attrs={'align': 'left'})
            
            event_abbreviation = first_row[0].find('a').text
            event_name = first_row[1].text
            
            event_dates = second_row[0].text
            event_place = second_row[1].text
            event_deadline = second_row[2].text
            
            df = pd.concat([df, pd.DataFrame([[event_abbreviation, event_name, event_dates, event_place, event_deadline]], columns=['abbreviation', 'name', 'dates', 'place', 'deadline'])], ignore_index=True)

    # Create 'country' column
    df['country'] = df['place'].apply(lambda x: 'online' if 'Virtual' in x or 'Online' in x else x.split(',')[-1].strip())
    
    # If country is 'N/A', remove the row
    df = df[df['country'] != 'N/A']

    return df

# # Usage
keyword = 'computer%20science'
df = scrape_wikicfp(keyword, pages=10)
df

# Analyze distribution of places
place_distribution = df['country'].value_counts()
print("Top Countries:\n", place_distribution.head())
print("\nBottom Countries:\n", place_distribution.tail())
############# SOLUTION END ###############

Top Countries:
 Australia      27
online         26
Switzerland    16
China          16
UAE            13
Name: country, dtype: int64

Bottom Countries:
 USA                       2
Indonesia                 2
Netherlands and online    1
India                     1
Algiers Algeria           1
Name: country, dtype: int64


### *Concepts required to complete this task:*

- Navigating through HTML code using functions
- DataFrame Creation
- Adding new columns to an existing DataFrame
- String operations


## Rubric

- +17 points for correct output
- +3 points for concise and logical code
- +3 points for ethical and mindful scraping
- +2 points for comments and logical variable names